In [41]:
from math import log, sqrt, exp
import time
import datetime as dt                   # date objects
import numpy as np                      # array manipulation
import matplotlib.pyplot as plot        # plotting
import pandas as pd                     # data analysis
import pandas_datareader as pdr
from scipy.stats import norm  # normal cdf

from ib_insync import *
util.startLoop()

#### Trading Assignment 1

Go to https://finance.yahoo.com and download adjusted close daily prices (end of trading day prices adjusted for dividends and splits) for the period 03/15/2022 to 09/15/2022 for SPY ETF.

In [42]:
# pull data from 
source = 'yahoo'
indicators = ['SPY']
end_date = dt.date.today()
start_date = end_date - dt.timedelta(days=365)
raw_data = pdr.DataReader(indicators, source, start_date, end_date)
data = raw_data['Adj Close']

1. Calculate the annual volatility for the S&P500 index. Use 252 trading days.

In [43]:
std_SPY = data.tail(252).SPY.std()
mean_SPY = data.tail(252).SPY.mean()
print(f"Annual Volatility SPY: {std_SPY}")

Annual Volatility SPY: 29.373467643093235


2. Go to https://www.cboe.com/delayed_quotes/spy/quote_table and select five option contracts that are nearest to ATM with expiration as close as possible to 30 days and have open interest and volume of at least 1,000 contracts. Alternatively, connect to IB and download the option chain. (this part can be done with a bot)
Using the volatility, you calculated in (1) and the Black-Scholes model, compute the “theoretical” prices of these call options.

In [44]:
try:
    ib.disconnect()
    time.sleep(5)
except:
    print("no ib connection to disconnect")

# initialize connection to IBKR
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)  # IB Trader Workstation
#ib.connect('127.0.0.1', 4002, clientId=1)    # IB Gateway

<IB connected to 127.0.0.1:7497 clientId=1>

In [45]:
ib.positions()

[Position(account='DU6066633', contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), position=117.0, avgCost=387.48679485),
 Position(account='DU6066633', contract=Option(conId=564970818, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=390.0, right='C', multiplier='100', currency='USD', localSymbol='SPY   221021C00390000', tradingClass='SPY'), position=-10.0, avgCost=1255.26991)]

In [46]:
# create a contract for the underlying s&p500 index
spx = Stock('SPY', 'SMART', 'USD')
ib.qualifyContracts(spx)
ib.reqMarketDataType(4)

# grab the ticker
[ticker] = ib.reqTickers(spx)
spx_price = ticker.marketPrice()

In [47]:
# generate the option chain
opt_chain = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
chain = next(c for c in opt_chain if c.tradingClass == 'SPY' and c.exchange == 'SMART')

# filter strikes for those close to in the money
strikes = [s for s in chain.strikes
           if spx_price - 20 < s < spx_price + 20]

expirations = sorted(exp for exp in chain.expirations)[:3]
expirations = ['20221021']

rights = ['C']

contracts = [Option('SPY', expiration, strike, right, 'SMART', tradingClass='SPY')
        for right in rights
        for expiration in expirations
        for strike in strikes]
contracts = ib.qualifyContracts(*contracts)
ib.reqMarketDataType(4)

In [48]:
# grab an example contract
call = contracts[0]
[ticker] = ib.reqTickers(call)
ticker

Ticker(contract=Option(conId=565285904, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=345.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   221021C00345000', tradingClass='SPY'), time=datetime.datetime(2022, 9, 30, 14, 45, 12, 402269, tzinfo=datetime.timezone.utc), marketDataType=3, bid=22.56, bidSize=6.0, ask=22.59, askSize=81.0, last=21.1, lastSize=1.0, volume=1.0, high=21.1, low=21.1, close=22.86, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.31435788076266324, delta=0.7793832143585, optPrice=22.559999465942383, pvDividend=0.0, gamma=0.01077165145770374, vega=0.2624713670578913, theta=-0.21592657834745604, undPrice=364.0249938964844), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.3154902820577127, delta=0.7786382969470584, optPrice=22.59000015258789, pvDividend=0.0, gamma=0.010753683392364025, vega=0.262501514397929, theta=-0.2169704225146334, undPrice=364.0249938964844), lastGreeks=OptionComputation(tickAttrib=0, 

In [49]:
# now calculate the theoretical prices of these options using BS

def bs_price(S, K, T, r, sigma, opt="c"):
    """Price an option with the black-scholes model

    Args:
        S (float): Price of the underlying security
        K (float): Strike price
        T (float): time to maturity
        r (float): risk free interest rate
        sigma (float): volatility of the underlying security
        opt (str, optional): option type. Defaults to "C" (call)

    Returns:
        float: the black-scholes price of the option
    """
    d1 = (np.log(S / K) + (r + sigma**2 / 2.0) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if opt == "c":
        bs_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif opt == "p":
        bs_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        bs_price = None
    return bs_price


Taking an example contract, we can compare the price with the Black-Scholes price. Do this for each contract in your list and find the one with the biggest discrepancy (hint: list comprehension is useful here!). For our purposes we'll just grab the first one.

In [50]:
price = spx_price
strike = ticker.contract.strike
maturity = 37/365
rfr = 0.03  # use shortterm libor!
vol = std_SPY / mean_SPY

opt_price = bs_price(price, strike, maturity, rfr, vol)

print(f"The close price of the option is: {ticker.close}")
print(f"The black-scholes price of the option is: {opt_price}")

The close price of the option is: 22.86
The black-scholes price of the option is: 20.060267977088586


**Note:** We will make parts 3-4 reusable by wrapping what we have done so far into a class that we can periodically instantiate and run it from [ta1.py](./ta1.py)

The following steps outline how to rebalance periodically after the initial sell order has been made.


3. At the beginning of the first trading day, sell 10 contracts of the call option with the largest mispricing. What would the proceeds from your trade be if the market price equals the BS option value? What are the actual proceeds from your trade?

In [51]:
ib.positions()

[Position(account='DU6066633', contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), position=117.0, avgCost=387.48679485),
 Position(account='DU6066633', contract=Option(conId=564970818, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=390.0, right='C', multiplier='100', currency='USD', localSymbol='SPY   221021C00390000', tradingClass='SPY'), position=-10.0, avgCost=1255.26991)]

4. As soon as possible after executing the trade, you initiate a delta hedge for your options. The hedge will be rebalanced (adjusted) during the next four weeks, i.e., until the expiation date or until the hedged position is closed. \n At initiation of your hedge, what is the delta for the call option? How many shares do you have to trade? Considering the proceeds from the options sale, how much would you have to you have to borrow (in real life) to finance a delta-neutral position. What is the interest cost incurred at the end of the first trading day?

Alright, let's delta hedge! The option delta is simply the sensitivity of the option to a change in price in the underlying security. Mathematically that is represented as...

$$
delta = \frac{\partial V}{\partial S}
$$

So we can simply grab the current delta from interactive using "modelGreeks" once we get the ticker...

In [52]:
# get current positions
positions = [p for p in ib.positions() if p.contract.symbol == "SPY"]

# ensure that you have an open option position
opts = [p for p in positions if p.contract.secType == "OPT"]
if not opts:
    print("No option position to hedge")
option = opts[0]
con = option.contract

contracts = ib.qualifyContracts(con)
ib.reqMarketDataType(4)
[ticker] = ib.reqTickers(*contracts)
ticker

Ticker(contract=Option(conId=564970818, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=390.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   221021C00390000', tradingClass='SPY'), time=datetime.datetime(2022, 9, 30, 14, 45, 15, 93147, tzinfo=datetime.timezone.utc), marketDataType=3, bid=1.18, bidSize=1070.0, ask=1.19, askSize=5295.0, last=1.15, lastSize=3.0, volume=2415.0, high=1.28, low=1.02, close=1.36, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.23534505471323391, delta=0.12435975368004751, optPrice=1.1799999475479126, pvDividend=0.0, gamma=0.009981165394986957, vega=0.19800914881600318, theta=-0.10426069997993533, undPrice=364.1499938964844), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.23596175869259947, delta=0.12493943081185128, optPrice=1.190000057220459, pvDividend=0.0, gamma=0.009990124757623951, vega=0.20068868830228914, theta=-0.10489584789466631, undPrice=364.1499938964844), lastGreeks=OptionComputation

In [53]:
# get the option delta from IB
delta = ticker.modelGreeks.delta
delta_neutral_pos = round(-option.position * delta * 100)

# calculate the difference between the hedge requirement and the current position
stks = [p for p in positions if p.contract.secType == "STK"]
if stks:
    hedge = delta_neutral_pos - stks[0].position
else:
    hedge = delta_neutral_pos
print(f"The deltahedge trade needed is SPY: {hedge}")

The deltahedge trade needed is SPY: 8.0


Peer closed connection.


In [ ]:
# TODO: only execute the trade if the hedge outweighs the risk
if abs(hedge) > 0:
    if hedge > 0:
        action = "BUY"
    else:
        action = "SELL"
        
    # return contract and order required to remain delta neutral
    contract = Stock("SPY", "SMART", "USD")
    ib.qualifyContracts(contract)
    order = MarketOrder(action, abs(hedge))
    
    # place trade
    trade = ib.placeOrder(contract, order)
    assert order in ib.orders()
    assert trade in ib.trades()
    while not trade.isDone():
        ib.waitOnUpdate()
else:
    print("Already delta-neutral, no trade required.")

In [ ]:
ib.positions()

5. At least twice during each trading day for the next four weeks, rebalance your portfolio so that your risk exposure remains as close to delta neutral as possible. Remember transaction costs! Do not trade unless the stock has moved enough, i.e. more than a round-trip transaction costs!  (this part can be done with an algo)

6. In four weeks, the options are either exercised or expire with 0 value or the hedged position is closed. What is the value of your delta neutral position? Did you make or lost money? Explain why?

In [ ]:
ib.disconnect()

Peer closed connection.
